**Libary**

To better work with `Google earth engine` in Python, we need to load the following packages (Noted: If you have not installed it, please see this [website](https://www.youtube.com/watch?v=Z_Kxg-EYvxM) on how to install Python packages).

In [1]:
import ee
import geemap as emap

In [2]:
# Initialize ee
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWiFJTQQbrxH4GI_aCKEtPpqUW_57KJeF1a3uNbXtiGRRxE5qx-tdvU

Successfully saved authorization token.


In [3]:
Map=emap.Map()
# Map.add_basemap("Google Satellite")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [17]:
# Map.addLayerControl()
# Map

# <font color='blue'>Working with Single Image </font> 

**Load images**

In [5]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
# Display the image
Map.centerObject(image,8)

Map.addLayer(image, {'bands':["B5","B4","B3"], 'min':0, 'max':0.5},"Landsat")

Map

Map(bottom=51101.0, center=[37.471602850590656, -122.14451407746262], controls=(WidgetControl(options=['positi…

In [ ]:
image.bandNames().getInfo()

##### <font color='blue'>Math Operations</font> 

**Calcualte NDVI**

In [7]:
# The hard way

NDVI=image.select("B5").subtract(image.select("B4")).divide(image.select("B5")).add(image.select("B4"))
# Visualizing NDVI
# palette = ['#0615ff', '#101eff', '#94ff85', '#19ff12']
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

Map.addLayer(NDVI,{'palette':palette, "min":-1.13, "max":0.93}, "NDVI")
Map

Map(bottom=25701.0, center=[37.471602850590656, -122.14451407746262], controls=(WidgetControl(options=['positi…

In [8]:
# We can also rename it
NDVI=NDVI.rename("NDVI")

print(NDVI.bandNames().getInfo())

['NDVI']


**Better Way of calculating NDVI or any indices**

In [13]:
EVI = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
})

# palette = ['#0615ff', '#101eff', '#94ff85', '#19ff12']
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

Map.addLayer(EVI,{'palette':palette, "min":-1.13, "max":0.93}, "EVI")
Map

Map(bottom=13067.133339219834, center=[36.88840804313823, -122.40966796875001], controls=(WidgetControl(option…


##### <font color="blue">Relational, conditional and Boolean operations</font>

This can be done using `lt` (less than), `gt` (greater than) or `eq` (equal), `gte`, `lte` (less than or equal). This function to create a binary image where `0` represents `False` and `1` represents `True`. We then can mask out `0` by using `updateMask()` function.

In [14]:
# Load NDVI 
NDWI = image.normalizedDifference(['B3', 'B5'])

Map.addLayer(NDWI,{'palette':palette, "min":-1.13, "max":0.93}, "NDVI")
Map

Map(bottom=13067.0, center=[36.88840804313823, -122.40966796875001], controls=(WidgetControl(options=['positio…

In [17]:
# Create a bare ground and water
water=NDVI.lt(0.2)# Create a binary image where <0.2 represents 1 and >0.2 represents 0.
# Map.addLayer(water,{},"Water")
# Mask out everything greater than 0.1
mask=water.updateMask(water)
# Set up color palette
palette = ['#0615ff'] # The 
Map.addLayer(mask,{"palette":palette}, "Water")
Map

Map(bottom=25813.46665410699, center=[36.98061506528613, -121.89880371093751], controls=(WidgetControl(options…

- We can combine `And()` `Or()` or `Not()` in the same image or with other images.

In [18]:
# We can also combine with And 
bare=NDVI.lt(0.1).And(NDWI.lt(0.5))
# Mask out the non-bare
mask=bare.updateMask(bare)
palette = ['#0615ff']
Map.addLayer(mask,{"palette":palette, min:0, max:1}, "Bare")
Map

Map(bottom=25805.0, center=[37.01571219880126, -122.26684570312501], controls=(WidgetControl(options=['positio…

**Create three classes representing three zones** using `.add()`

In [20]:
# Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))
Map.addLayer(zones, {}, 'development zones')

# # Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones')
Map

Map(bottom=25816.0, center=[48.8683, 2.373], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [22]:
nl2012.get("system:band_names").getInfo()

['avg_vis', 'stable_lights', 'cf_cvg', 'avg_lights_x_pct']

**Where**

`where()` return a binary image or image with replaced value if condition meets.

In [4]:
# Load a cloudy Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130603')

# Load another image to replace the cloudy pixels.
replacement = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130416')
Map.addLayer(replacement,{'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5},"Original Image")
# Compute a cloud score band.
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')

# Set cloudy pixels to the other image.
new_image = image.where(cloud.gt(5), replacement)

# # Display the result.
Map.centerObject(replacement, 9)
Map.addLayer(new_image,
             {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5},
             'clouds replaced')

# Map.addLayerControl()
Map

Map(bottom=1851.0, center=[37.471642191576414, -122.11085841101837], controls=(WidgetControl(options=['positio…

#### <font color='blue'>Spectral Transformation</font> 

In [ ]:
# Convert the RGB bands to the HSV color space.
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

# Swap in the panchromatic band and convert back to RGB.
sharpened = ee.Image.cat([
  hsv.select('hue'), hsv.select('saturation'), image.select('B8')
]).hsvToRgb()

# Display the pan-sharpened result.
Map.setCenter(-122.44829, 37.76664, 13)
# Map.addLayer(sharpened,
#              {'min': 0, 'max': 0.25, 'gamma': [1.3, 1.3, 1.3]},
#              'pan-sharpened')

# Map.addLayerControl()
# Map

# <font color="blue"> Export a single image </font>

In [11]:
# Obtain Landsat 8 TOA Level 1
LS8=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2018-01-01","2018-12-31")\
.filterMetadata("CLOUD_COVER","less_than",10).filterBounds(Singapore) # Filter boundary of Singapore and cloud cover
Map.addLayer(LS8,{},"LS8")
Map

Map(bottom=65340.0, center=[1.3621763466641712, 103.82080078125001], controls=(WidgetControl(options=['positio…

In [12]:
# Display the Hanoi boundary 
UN=ee.FeatureCollection("FAO/GAUL/2015/level2")

Singapore=UN.filterMetadata("ADM0_NAME","equals","Singapore")

Map.centerObject(Singapore,9)
Map.addLayer(ee.Image().paint(Singapore,0,1.5),{"palette":["red"]},"Singapore") # Colour = 0 or set color="red", and width =1.5
Map

Map(bottom=260617.0, center=[1.3611355787659067, 103.8202830927031], controls=(WidgetControl(options=['positio…

In [13]:
# Obtain median over the Image Collection
LS_Sing=LS8.median().float().clip(Singapore) # Convert to float32 as it has both float32 and float64. Can't download if mixed
Map.addLayer(LS_Sing,{},"LSSIng")
Map

Map(bottom=65340.0, center=[1.3611355787659067, 103.8202830927031], controls=(WidgetControl(options=['position…

In [ ]:
# Download a single image with boundary of Singapore and 30m pixel size
task = ee.batch.Export.image.toDrive(image=LS_Sing,  # an ee.Image object.
                                     region=Singapore.geometry().bounds().getInfo()["coordinates"],  # an ee.Geometry object.
                                     description="LS_Sing",
                                     folder="Examples",
                                     fileNamePrefix="LS_Sing",
                                     scale=30,
                                     crs="EPSG:4326")
task.start()

In [14]:
task.status()

NameError: name 'task' is not defined